In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
"""Atributos previsores (x)"""
x = pd.read_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x.csv')

In [ ]:
"""Função para preencher as colunas das classes (y) com base no diagnóstico"""
def binary(row):
    if row['diagnosis'] == 'M':
        return 1
    return 0

In [ ]:
"""Criando o arquivo binário"""
y = x.apply(binary, axis=1)
y = pd.DataFrame(y, columns=['binary'])
y.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y.csv', index=False)
x.drop(['id', 'diagnosis'], axis='columns', inplace=True)

In [ ]:
x_training, x_test, y_training, y_test = train_test_split(x, y, test_size=0.25) # 25% para testar e 75% para treinar
x_training.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x_training.csv')
y_training.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y_training.csv')
x_test.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x_test.csv')
y_test.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y_test.csv')

In [ ]:
"""Criando a variável que representa a rede neural"""
neural_net = keras.Sequential([
    keras.layers.InputLayer(shape=(10,)), # shape deve ser igual ao número de atributos previsores
    keras.layers.Dense(units=6, activation='relu', kernel_initializer='random_uniform'), # Rede neural densa. Todos os neurônios de uma camada estão conectados aos neurônios da próxima camada. Para definir o parâmetro units, fazemos a média entre o número de neurônios de entrada e de saída, sempre arredondando para cima
    # Utilizamos a ReLU pois estamos criando a camada oculta
    # kernel_initializer é a forma como os pesos serão inicializados
    keras.layers.Dense(units=1, activation='sigmoid') # Camada de saída
])

In [ ]:
neural_net.summary() # (66) Param = 6 [neurônios da camada oculta] * 10 [número de parâmetros] + 6 [bias]
# (7) Param = 6 [neurônios da camada oculta] + 1 [bias]
# (73) Total params = 66 + 7. Representam os pesos que serão ajustados

In [ ]:
"""Definindo a maneira de ajuste dos pesos, função que calcula o erro, e as métricas retornadas"""
neural_net.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
# optimizer = Algoritmo utilizado para o ajuste dos pesos. Adam é uma melhoria do Sthocastic Gradient Descent
# loss = Função que calcula o erro. BinaryCrossentropy para duas classes. CategoricalCrossentropy para mais de duas classes. MeanAbsolute e MeanSquare são utilizadas em regressões (previsões de números). As funções baseadas em Crossentropy utilizam cálculos de logaritmos, sendo mais otmizadas e tendo um desempenho melhor
# metrics = Lista com as métricas que serão retornadas. BinaryAccuracy para problemas de classificação binária

In [ ]:
"""Treinando o modelo"""
neural_net.fit(x_training, y_training, batch_size=10, epochs=100)
# batch_size = Quantidade de dados que vai ser enviada por vez para a rede neural. Os pesos são atualizados após o processamento de um batch que, nesse caso, é composto por 10 registros
# epochs = Quantidade de vezes que o processo será executado. Em cada uma das vezes, a rede neural irá processar todos os registros

In [ ]:
"""Previsões e avaliação da base de dados de teste"""
predictions = neural_net.predict(x_test)
predictions = predictions > 0.5 # Converte os valores retornados pela função sigmoide (valores estes que estão entre 0 e 1) para True ou False. Quanto maior o valor, maior a confiança que esperamos do algoritmo
predictions_df = pd.DataFrame(predictions, columns=['predictions'])
predictions_df.to_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/predictions.csv')

In [ ]:
"""Avaliando a acurácia do modelo"""
accuracy_score(y_test, predictions)

In [ ]:
"""Visualizando a matriz de erro"""
confusion_matrix(y_test, predictions)